# Data exploration

In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://macbookpro.homenet.telecomitalia.it:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1753775449322)
SparkSession available as 'spark'


import org.apache.spark


### Schema definitions

In [2]:
import org.apache.spark.sql.types._

val reviewSchema = StructType(
  Seq(
    StructField("user_id",  StringType,            nullable = true),
    StructField("name",     StringType,            nullable = true),
    StructField("time",     LongType,              nullable = false),
    StructField("rating",   DoubleType,            nullable = true),
    StructField("text",     StringType,            nullable = true),
    StructField("pics",     ArrayType(StringType), nullable = true),
    StructField("resp",     StructType(
      Seq(
        StructField("time", LongType,              nullable = false),
        StructField("text", StringType,            nullable = true)
      )
    ),                                             nullable = true),
    StructField("gmap_id",  StringType,            nullable = false),
  )
)

case class Response(time: Long, text: Option[String])

case class Review(
  user_id: Option[String],
  name: Option[String],
  time: Long,
  rating: Option[Double],
  text: Option[String],
  pics: Seq[String],
  resp: Option[Response],
  gmap_id: String
)

import org.apache.spark.sql.types._
reviewSchema: org.apache.spark.sql.types.StructType = StructType(StructField(user_id,StringType,true),StructField(name,StringType,true),StructField(time,LongType,false),StructField(rating,DoubleType,true),StructField(text,StringType,true),StructField(pics,ArrayType(StringType,true),true),StructField(resp,StructType(StructField(time,LongType,false),StructField(text,StringType,true)),true),StructField(gmap_id,StringType,false))
defined class Response
defined class Review


In [3]:
val metadataSchema = StructType(
  Seq(
    StructField("name",             StringType,                                 nullable = true),
    StructField("address",          StringType,                                 nullable = true),
    StructField("gmap_id",          StringType,                                 nullable = false),
    StructField("description",      StringType,                                 nullable = true),
    StructField("latitude",         DoubleType,                                 nullable = false),
    StructField("longitude",        DoubleType,                                 nullable = false),
    StructField("category",         ArrayType(StringType),                      nullable = true),
    StructField("avg_rating",       DoubleType,                                 nullable = false),
    StructField("num_of_reviews",   IntegerType,                                nullable = false),
    StructField("price",            StringType,                                 nullable = false),
    StructField("hours",            ArrayType(ArrayType(StringType)),           nullable = true),
    StructField("MISC",             MapType(StringType, ArrayType(StringType)), nullable = false),
    StructField("state",            StringType,                                 nullable = true),
    StructField("relative_results", ArrayType(StringType),                      nullable = true),
    StructField("url",              StringType,                                 nullable = false),
  )
)

case class Metadata(
  name: Option[String],
  address: Option[String],
  gmap_id: String,
  description: Option[String],
  latitude: Double,
  longitude: Double,
  category: Seq[String],
  avg_rating: Double,
  num_of_reviews: Int,
  price: String,
  hours: Seq[Seq[String]],
  MISC: Map[String, Seq[String]],
  state: Option[String],
  relative_results: Seq[String],
  url: String
)

metadataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructField(address,StringType,true),StructField(gmap_id,StringType,false),StructField(description,StringType,true),StructField(latitude,DoubleType,false),StructField(longitude,DoubleType,false),StructField(category,ArrayType(StringType,true),true),StructField(avg_rating,DoubleType,false),StructField(num_of_reviews,IntegerType,false),StructField(price,StringType,false),StructField(hours,ArrayType(ArrayType(StringType,true),true),true),StructField(MISC,MapType(StringType,ArrayType(StringType,true),true),false),StructField(state,StringType,true),StructField(relative_results,ArrayType(StringType,true),true),StructField(url,StringType,false))
defined class Metadata


### Dataset load and parse

In [4]:
import java.nio.file.Paths

val projectDir = Paths.get(System.getProperty("user.dir")).getParent.getParent.getParent
val reviewsPath = s"$projectDir/dataset/sample-reviews.ndjson"
val metadataPath = s"$projectDir/dataset/metadata.ndjson"

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("NDJSON Reader")
  .master("local[*]") // Needed in local mode
  .getOrCreate()

val reviewsDf = spark.read
  .schema(reviewSchema)
  .json(reviewsPath)
  .withColumn("pics", when (col("pics") isNull, array()) otherwise col("pics"))
  .as[Review]

val metadataDf = spark.read
  .schema(metadataSchema)
  .json(metadataPath)
  .withColumn("category", when (col("category") isNull, array()) otherwise col("category"))
  .withColumn("hours", when (col("hours") isNull, array()) otherwise col("hours"))
  .withColumn("relative_results", when (col("relative_results") isNull, array()) otherwise col("relative_results"))
  .as[Metadata]

reviewsDf.printSchema()
metadataDf.printSchema()

val reviewsRdd = reviewsDf.rdd.map(Review.unapply).map(_.get)
val metaRdd = metadataDf.rdd.map(Metadata.unapply).map(_.get)

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resp: struct (nullable = true)
 |    |-- time: long (nullable = true)
 |    |-- text: string (nullable = true)
 |-- gmap_id: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- avg_rating: double (nullable = true)
 |-- num_of_reviews: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (co

import java.nio.file.Paths
projectDir: java.nio.file.Path = /Users/lucatassi/Projects/big-data/big-data-project
reviewsPath: String = /Users/lucatassi/Projects/big-data/big-data-project/dataset/sample-reviews.ndjson
metadataPath: String = /Users/lucatassi/Projects/big-data/big-data-project/dataset/metadata.ndjson
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@64bc9df9
reviewsDf: org.apache.spark.sql.Dataset[Review] = [user_id: string, name: string ... 6 more fields]
metadataDf: org.apache.spark.sql.Dataset[Metadata] = [name: string, address: string ... 13 more fields]
reviewsRdd: org.apache.spark.rdd.RDD[(Option[String], Option[String], Long, Option[Double], Option[String], Seq[String], Option[Response], String)] = M...


## Exploration

---

**Metadata**: (name, address, <ins>gmap_id</ins>, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url)

**Review**: (user_id, name, time, rating, text, pics, responses, <ins>gmap_id</ins>)

---

In [5]:
reviewsRdd
  .filter(_._2.contains("Hossein"))
  .collect()

res0: Array[(Option[String], Option[String], Long, Option[Double], Option[String], Seq[String], Option[Response], String)] = Array((Some(113587357589866776535),Some(Hossein),1617400126306,Some(5.0),None,List(),None,0x808fbb350a474eb9:0x20e475f4d549a0b7), (Some(113587357589866776535),Some(Hossein),1573824465526,Some(4.0),None,List(),None,0x808fc76a4f691325:0x530bc2080d135950))


In [6]:
metaRdd
  .filter(_._3 == "0x80dcdbd91ac0ff97:0x40cb80cf24283e4d")
  .collect()

res1: Array[(Option[String], Option[String], String, Option[String], Double, Double, Seq[String], Double, Int, String, Seq[Seq[String]], Map[String,Seq[String]], Option[String], Seq[String], String)] = Array((Some(Shell),Some(Shell, 395 W First St, Tustin, CA 92780),0x80dcdbd91ac0ff97:0x40cb80cf24283e4d,None,33.7459242,-117.82758,List(Gas station, Alternative fuel station, ATM, Convenience store, Electric vehicle charging station),4.0,34,$$,List(),Map(Accessibility -> List(Wheelchair accessible entrance)),Some(Open now),List(0x80dcdbe3deac39db:0x624e68a174a293b7, 0x80dcd95ef4e83e17:0xb0a6b7bf134626d5, 0x80dcdc214dd2ddc5:0x933178bf1e6f113a, 0x80dd2674301aa6ab:0x7dfb81933c80facb, 0x80dcda2ad3aee46b:0x9de6be25417d4cb),https://www.google.com/maps/place//data=!4m2!3m1!1s0x80dcdbd91ac0ff97:0x...


In [7]:
// beware `rating` can be empty!!
reviewsRdd.filter(_._4.isEmpty).count()

res2: Long = 9745


In [8]:
// beware `user_id` can be empty!!
reviewsRdd.filter(_._1.isEmpty).count()

res3: Long = 9745


How many distinct businesses? 1.228.804

In [9]:
metaRdd.map(_._3).distinct().count()

res4: Long = 1228804


How many average ratings per user? [Incomplete data since we are using a sample of the dataset]

In [10]:
val avgRatingsPerUser = reviewsRdd
  .filter(_._1.isDefined) // user_id can be not defined
  .map(_._1.get -> 1)
  .reduceByKey(_ + _) // [(user_id, #ratings written by user_id)*]
  .aggregate((0, 0))((acc, v) => (acc._1 + v._2, acc._2 + 1), (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2))
"Average ratings per user: " + avgRatingsPerUser._1.toDouble / avgRatingsPerUser._2

avgRatingsPerUser: (Int, Int) = (1696279,1422429)
res5: String = Average ratings per user: 1.1925227902412001


How many average ratings per business? [Incomplete data since we are using a sample of the dataset]

In [11]:
val avgRatingsPerBusiness = reviewsRdd
  .map(_._8 -> 1)
  .reduceByKey(_ + _) // [(gmap_id, #ratings for gmap_id)*]
  .aggregate((0, 0))((acc, v) => (acc._1 + v._2, acc._2 + 1), (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2))
"Average ratings per business: " + avgRatingsPerBusiness._1.toDouble / avgRatingsPerBusiness._2

avgRatingsPerBusiness: (Int, Int) = (1706024,486237)
res6: String = Average ratings per business: 3.508626451709763


Average response rate? [Incomplete data since we are using a sample of the dataset]

In [15]:
val avgResponseRate = reviewsRdd
  .map(r => r._8 -> (if (r._7.isDefined) 1 else 0))
  .aggregate((0, 0))((acc, v) => (acc._1 + v._2, acc._2 + 1), (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2))
"Average response rate: " + (avgResponseRate._1.toDouble / avgResponseRate._2) * 100 + " %"

avgResponseRate: (Int, Int) = (218759,1706024)
res10: String = Average response rate: 12.8227387187988 %
